In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [15]:
ad_data = pd.read_csv('advertising.csv')

In [16]:
X = ad_data[['Daily Time Spent on Site', 'Age', 'Area Income','Daily Internet Usage', 'Male']]
y = ad_data['Clicked on Ad']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### First way 

In [14]:
import numpy as np
def sigmoid(z):
    
    # convert input to a numpy array
    z = np.array(z)
    
    # You need to return the following variables correctly 
    g = np.zeros(z.shape)

    # ====================== YOUR CODE HERE ======================
    
    g+=1/(1+np.exp(-z))

    

    # =============================================================
    return g

In [18]:
# Setup the data matrix appropriately, and add ones for the intercept term
m, n = X_train.shape

# Add intercept term to X
X_train_1 = np.concatenate([np.ones((m, 1)), X_train], axis=1)
X_train_1[0]

array([1.000000e+00, 7.765000e+01, 2.800000e+01, 6.619866e+04,
       2.080100e+02, 0.000000e+00])

In [19]:
def costFunction(theta, X, y):
    
    # Initialize some useful values
    m = y.size  # number of training examples

    # You need to return the following variables correctly 
    J = 0
    grad = np.zeros(theta.shape)

    # ====================== YOUR CODE HERE ======================
    J = (-1/m) * ((np.sum(y*(np.log(1/(np.exp(-np.dot(theta,X.transpose())) + 1))))) + 
                     (np.sum((1-y)*(np.log(1-(1/(np.exp(-np.dot(theta,X.transpose())) + 1)))))))
    
    for i in range(n+1):
        grad[i] = (1/m) * np.sum(((1/(1 + np.exp(-np.dot(theta,X.transpose())))) - y) * X[:,i])
    
    # =============================================================
    return J, grad

In [22]:
# Initialize fitting parameters
initial_theta = np.zeros(n+1)

cost, grad = costFunction(initial_theta, X_train_1, y_train)

print('Cost at initial theta (zeros): {:.3f}'.format(cost))



Cost at initial theta (zeros): 0.693


In [25]:
from scipy import optimize
# set options for optimize.minimize
options= {'maxiter': 400}

# see documention for scipy's optimize.minimize  for description about
# the different parameters
# The function returns an object `OptimizeResult`
# We use truncated Newton algorithm for optimization which is 
# equivalent to MATLAB's fminunc
# See https://stackoverflow.com/questions/18801002/fminunc-alternate-in-numpy
res = optimize.minimize(costFunction,
                        initial_theta,
                        (X_train_1, y_train),
                        jac=True,
                        method='TNC',
                        options=options)

# the fun property of `OptimizeResult` object returns
# the value of costFunction at optimized theta
cost = res.fun

# the optimized theta is in the x property
theta = res.x

# Print theta to screen
print('Cost at theta found by optimize.minimize: {:.3f}'.format(cost))


print('theta:')
print('\t[{:.3f}, {:.3f}, {:.3f},{:.3f}, {:.3f}, {:.3f}]'.format(*theta))

Cost at theta found by optimize.minimize: 0.080
theta:
	[30.030, -0.194, 0.169,-0.000, -0.072, -0.887]


In [27]:
def predict(theta, X):
    
    m = X.shape[0] # Number of training examples

    p = np.zeros(m)
   
    temp = sigmoid(np.dot(theta,X.transpose()))
    #p = [1 for i in range(m) if temp[i]>=0.5]
    for i in range(m):
        if temp[i] >= 0.5:
            p[i] = 1
        else:
            p[i] = 0
 
    return p

In [30]:
# Setup the data matrix appropriately, and add ones for the intercept term
m, n = X_test.shape

# Add intercept term to X
X_test_1 = np.concatenate([np.ones((m, 1)), X_test], axis=1)
X_test_1[0]

array([1.000000e+00, 6.326000e+01, 2.900000e+01, 5.478737e+04,
       1.204600e+02, 1.000000e+00])

In [35]:
#  Predict probability for a student with score 45 on exam 1 
#  and score 85 on exam 2 
prob = sigmoid(np.dot(X_test_1[0], theta))
print('For X_test_1[0] with y_test[0]=1,'
      'we predict : {:.3f}'.format(prob))

# Compute accuracy on our training set
p = predict(theta, X_test_1)
print('Test Accuracy: {:.2f} %'.format(np.mean(p == y_test) * 100))


For X_test_1[0] with y_test[0]=1,we predict : 0.993
Test Accuracy: 96.67 %


In [42]:
print(classification_report(y_test,p))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       162
           1       0.98      0.96      0.97       168

    accuracy                           0.97       330
   macro avg       0.97      0.97      0.97       330
weighted avg       0.97      0.97      0.97       330



## Second way

In [36]:
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)

LogisticRegression()

In [37]:
predictions = logmodel.predict(X_test)

In [38]:
print('Test Accuracy: {:.2f} %'.format(np.mean(predictions == y_test) * 100))

Test Accuracy: 90.61 %


In [40]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.86      0.96      0.91       162
           1       0.96      0.85      0.90       168

    accuracy                           0.91       330
   macro avg       0.91      0.91      0.91       330
weighted avg       0.91      0.91      0.91       330

